# General Imports

**Importing all libraries**

In [1]:
import numpy as np
import pandas as pd
import warnings

import plotly.graph_objects as go
from bayes_opt import BayesianOptimization
import statsmodels.api as sm

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.inspection import permutation_importance


Disable warnings

In [2]:
# Disable all warnings
warnings.filterwarnings("ignore")

# Enable warnings again
## warnings.filterwarnings("default")

Generate train and test data

In [3]:
# Read Data
df = pd.read_csv("data_finish_prep.csv")

# Select only the usefull part of the df
df.drop(columns=['verschil_Lengte',
               'verschil_Gewicht', 'verschil_TUG',
               'verschil_BMI', 'verschil_Conditie',
               'verschil_Lenigheid', 'verschil_Knijpkracht'], inplace=True)

# Drop NaN values
df.dropna(inplace=True)

# Define X and y
X = df.drop(columns=['verschil_6 MWT'])
y = df['verschil_6 MWT']

# Split data into train & test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    regressor = model(**params)
    
    # Assuming you have X_train, y_train defined for regression
    scores = cross_val_score(regressor, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')
    return -scores.mean()

# MLR

**feature selection**

In [5]:
# Fit the Full Model
X =  sm.add_constant(X_train)  # Add constant term for the intercept
model = sm.OLS(y_train, X_train).fit()  # Fit the MLR model
best_model = model  # Initialize the best model
selected_features = X_train.columns.tolist()
best_features = X_train.columns.tolist()
best_features_rmse = np.sqrt(-cross_val_score(LinearRegression(), X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean())

# Iteratively remove one feature at a time based on p-values
while True:
    # Compute p-values
    p_values = model.pvalues[1:]  # Exclude the constant term

    # Identify Insignificant Features
    insignificant_feature = p_values.idxmax()
    max_p_value = p_values.max()

    # Remove Insignificant Feature
    X = X.drop(insignificant_feature, axis=1)
    selected_features.remove(insignificant_feature)

    # When there a no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    model = sm.OLS(y_train, X).fit()
    
    # Evaluate Model Performance
    rmse = np.sqrt(-cross_val_score(LinearRegression(), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error').mean())

    # Check if the model performs better without the insignificant features
    if rmse <= best_features_rmse:
        best_features_rmse = rmse
        best_features = selected_features.copy()
        best_model = model

# Save results
best_model_mlr = LinearRegression().fit(X_train[best_features], y_train)
best_features_mlr = best_features

# Show results
print("Best features:", best_features)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features)))

Best features: ['AfstandTotKinderdagverblijf', 'TotaalDiefstalUitWoningSchuurED', 'VernielingMisdrijfTegenOpenbareOrde', 'AfstandTotHotelED', 'AfstandTotSemiOpenbaarGroenTotaal', 'AfstandTotVerblijfsrecreatiefTerrein', 'AfstandTotBibliotheek', 'AfstandTotKunstijsbaan', 'AfstandTotMuseum', 'EenOfMeerLangdurigeAandoeningen', 'BeperkingInHoren', 'ZwaarBelasteMantelzorgers', 'Rokers']
Dropped features: ['KLFinancieleDienstenOnroerendGoed', 'AfstandTotZonnebank', 'AfstandTotApotheek', 'AfstandTotConsultatiebureau', 'BouwjaarVanaf2000', 'GeboorteRelatief', 'AfstandTotDagrecreatiefTerrein', 'AfstandTotBioscoop', 'AfstandTotHuisartsenpost', 'BeperkingInZien', 'AfstandTotOpenbaarGroenTotaal', 'PersonenautoSNaarOppervlakte', 'OppervlakteWater', 'AfstandTotPodiumkunstenTotaal', 'VoldoetAanBeweegrichtlijn', 'AfstandTotBrandweerkazerne', 'AfstandTotZwembad', 'AfstandTotAttractie', 'GemiddeldeWoningwaarde', 'SterfteRelatief', 'InBezitOverigeVerhuurders', 'AfstandTotCafetariaED', 'AfstandTotOpenNatTe

*Er zijn geen hyperparameters om te optimaliseren.*

# Support Vector Machines

**Hyperparameter optimalisatie**

In [6]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    classifier = model(**params)
    
    # Calculating accuracy based on cross-validation
    scores = cross_val_score(classifier, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
    return scores.mean()

params_ranges = {
    'C': (0.1, 10),
    'kernel_int': (1, 4),
    'gamma': (0.001, 0.1)
}

kernel_mapping = {
    1: 'linear',
    2: 'rbf',
    3: 'poly',
    4: 'sigmoid'
}

# Example usage with SVM
model = SVR
model_bo = BayesianOptimization(f=lambda C, kernel_int, gamma:
                                    bo_params_generic(model, {
                                        'C': C,
                                        'kernel': kernel_mapping[int(kernel_int)],
                                        'gamma': gamma
                                    }, X_train, y_train),
                             pbounds=params_ranges)

results = model_bo.maximize(n_iter=40, init_points=10)
params = model_bo.max['params']

# Creating a model with the best hyperparameters
best_model_svm = model(
    C=params['C'],
    kernel=kernel_mapping[int(params['kernel_int'])],
    gamma=params['gamma']
)

# Fit the model
best_model_svm.fit(X_train, y_train)


|   iter    |  target   |     C     |   gamma   | kernel... |
-------------------------------------------------------------


| 1         | -4.585e+0 | 0.7077    | 0.04572   | 3.822     |
| 2         | -4.529e+0 | 3.507     | 0.07      | 1.505     |
| 3         | -4.565e+0 | 8.62      | 0.02918   | 3.196     |
| 4         | -4.581e+0 | 3.089     | 0.03111   | 3.314     |
| 5         | -4.536e+0 | 4.407     | 0.06039   | 3.258     |
| 6         | -4.512e+0 | 7.862     | 0.04272   | 1.406     |
| 7         | -4.519e+0 | 5.927     | 0.06606   | 1.514     |
| 8         | -4.537e+0 | 2.304     | 0.0642    | 3.297     |
| 9         | -4.568e+0 | 5.6       | 0.03113   | 2.67      |
| 10        | -4.526e+0 | 7.733     | 0.06046   | 3.565     |
| 11        | -4.553e+0 | 8.997     | 0.06508   | 2.491     |
| 12        | -4.536e+0 | 1.304     | 0.08803   | 3.301     |
| 13        | -4.552e+0 | 0.3798    | 0.09835   | 3.553     |
| 14        | -4.513e+0 | 7.443     | 0.04111   | 1.125     |
| 15        | -4.513e+0 | 7.504     | 0.02537   | 1.53      |
| 16        | -4.516e+0 | 6.773     | 0.08387   | 1.331     |
| 17    

SVR(C=10.0, gamma=0.1, kernel='linear')

**Feature Selection**

In [7]:
# Get the accuracy score of the model with all features
best_accuracy = cross_val_score(best_model_svm, X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()
best_model = best_model_svm
selected_features = X_train.columns
best_features = selected_features

while len(selected_features) > 1:
    try:
        # Retrieve the coefficients from the SVM model
        coefficients = best_model_svm.coef_[0]

        # Sort features based on their coefficient magnitudes
        sorted_indices = np.argsort(np.abs(coefficients))
        selected_features = [X_train.columns[i] for i in sorted_indices[::-1]]

        # Remove the least important feature
        selected_features = selected_features[:-1]
        
        # Refit the linear SVM classifier
        best_model_svm_classification.fit(X_train[selected_features], y_train)
        
        # Evaluate Model Performance
        accuracy = cross_val_score(best_model_svm, X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

        # Check if the model performs better without the insignificant features
        if accuracy >= best_accuracy:
            best_accuracy = accuracy
            best_features = selected_features.copy()
            best_model = best_model_svm
    except:
        print("Scince the kernal is not linear, this feature selection is not possible") 
        break

# Save results
best_model_svm = best_model
best_features_svm = best_features

# Show results
print("Best features:", best_features_svm)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_svm)))

Scince the kernal is not linear, this feature selection is not possible
Best features: Index(['NederlandseAntillenEnAruba', 'GeboorteRelatief', 'SterfteRelatief',
       'GemiddeldeWoningwaarde', 'PercentageOnbewoond',
       'InBezitWoningcorporatie', 'InBezitOverigeVerhuurders',
       'EigendomOnbekend', 'BouwjaarVanaf2000', 'kPersonenMetLaagsteInkomen',
       'ALandbouwBosbouwEnVisserij', 'KLFinancieleDienstenOnroerendGoed',
       'PersonenautoSOverigeBrandstof', 'PersonenautoSPerHuishouden',
       'PersonenautoSNaarOppervlakte', 'AfstandTotHuisartsenpraktijk',
       'AfstandTotKinderdagverblijf', 'AfstandTotSchool', 'OppervlakteWater',
       'TotaalDiefstalUitWoningSchuurED',
       'VernielingMisdrijfTegenOpenbareOrde', 'GeweldsEnSeksueleMisdrijven',
       'AfstandTotHuisartsenpost', 'AfstandTotApotheek',
       'AfstandTotConsultatiebureau', 'AfstandTotOvDagelLevensmiddelen',
       'AfstandTotWarenhuis', 'AfstandTotCafeED', 'AfstandTotCafetariaED',
       'AfstandTotResta

# Random Forest

**Feature selection**

In [8]:
# Get the accuracy score of the model with all the features
rf = RandomForestRegressor(n_estimators=100, random_state=0).fit(X_train, y_train)
best_rf = rf
selected_features = X_train.columns.tolist()
best_features = selected_features
best_features_accuracy = cross_val_score(RandomForestRegressor(n_estimators=100, random_state=0), X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

while True:
    # Make DataFrame of feature impotances
    result = permutation_importance(
            rf, X_train[selected_features], y_train, n_repeats=10, random_state=42, n_jobs=-1
        )
    df_importances = pd.DataFrame({'feature': X_train[selected_features].columns, 'importance': result.importances_mean, 'std': result.importances_std})
    df_importances = df_importances.sort_values('importance')

    # Remove least important feature
    selected_features.remove(df_importances['feature'].iloc[0])

    # When there are no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    rf = RandomForestRegressor(n_estimators=100, random_state=0).fit(X_train[selected_features], y_train)

    # Evaluate Model Performance
    accuracy = cross_val_score(RandomForestRegressor(n_estimators=100, random_state=0), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

    # Check if the model performs better without the insignificant features
    if accuracy >= best_features_accuracy:
        best_features_accuracy = accuracy
        best_features = selected_features.copy()
        best_model = rf

# Save results
best_model_rf = best_model
best_features_rf = best_features

# Show results
print("Best features:", best_features_rf)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_rf)))

Best features: ['AfstandTotApotheek', 'AfstandTotBioscoop', 'AfstandTotAttractie']
Dropped features: ['KLFinancieleDienstenOnroerendGoed', 'AfstandTotZonnebank', 'AfstandTotConsultatiebureau', 'AfstandTotHotelED', 'BeperkingInHoren', 'BouwjaarVanaf2000', 'GeboorteRelatief', 'AfstandTotDagrecreatiefTerrein', 'AfstandTotHuisartsenpost', 'BeperkingInZien', 'AfstandTotOpenbaarGroenTotaal', 'PersonenautoSNaarOppervlakte', 'OppervlakteWater', 'AfstandTotPodiumkunstenTotaal', 'VoldoetAanBeweegrichtlijn', 'AfstandTotBrandweerkazerne', 'AfstandTotZwembad', 'AfstandTotBibliotheek', 'AfstandTotMuseum', 'GemiddeldeWoningwaarde', 'SterfteRelatief', 'InBezitOverigeVerhuurders', 'AfstandTotCafetariaED', 'AfstandTotOpenNatTerreinTotaal', 'Ondergewicht', 'kPersonenMetLaagsteInkomen', 'AfstandTotBuitenschoolseOpvang', 'AfstandTotCafeED', 'VoldoetAanRichtlijnAlcoholgebruik', 'AfstandTotBegraafplaats', 'PersonenautoSOverigeBrandstof', 'PersonenautoSPerHuishouden', 'AfstandTotOpenNatNatuurlijkTerrein', 'Ro

**Hyperparameter optimalisatie**

In [9]:
def bo_params_rf(min_samples_split,max_depth,max_features,n_estimators):
    
    params = {
        'min_samples_split': round(min_samples_split),
        'max_depth': round(max_depth),
        'max_features': max_features,
        'n_estimators':round(n_estimators)
    }    
        
    clf = RandomForestRegressor(min_samples_split=int(params['min_samples_split']), 
                                 max_depth=int(params['max_depth']), 
                                 max_features=params['max_features'],
                                 n_estimators=int(params['n_estimators']), 
                                 bootstrap=False,
                                 n_jobs = -1)
    return cross_val_score(clf, X_train[best_features_rf], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

# Create Bayesian Optimization
rf_bo = BayesianOptimization(f=bo_params_rf, pbounds={
    'n_estimators': (100, 1000),
    'max_depth': (1, 20),
    'max_features': (0.1, 1),
    'min_samples_split': (2, 12)
})

# Bayesian optimization
results = rf_bo.maximize(n_iter=40, init_points=10)

# Getting best hyperparameters from bayesian optimization
params = rf_bo.max['params']
params['n_estimators']= round(params['n_estimators'])
params['min_samples_split']= round(params['min_samples_split'])
params['max_depth']= round(params['max_depth'])

# Making random forest model with the best hyperparameters
# criterion entropy is after the hyperparameter optimization since it's more accurate but increases training time
best_model_rf_classification = RandomForestRegressor(min_samples_split=params['min_samples_split'],
                                                max_depth=params['max_depth'],
                                                max_features=params['max_features'],
                                                n_estimators=params['n_estimators'],
                                                bootstrap=False)

best_model_rf_classification.fit(X_train[best_features_rf], y_train)

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------


| 1         | -4.485e+0 | 15.99     | 0.4798    | 4.665     | 163.0     |
| 2         | -4.48e+03 | 8.102     | 0.2177    | 10.88     | 782.8     |
| 3         | -4.485e+0 | 10.36     | 0.9763    | 5.224     | 341.0     |
| 4         | -4.485e+0 | 13.99     | 0.9378    | 8.969     | 421.4     |
| 5         | -4.483e+0 | 19.36     | 0.8433    | 11.51     | 522.7     |
| 6         | -4.477e+0 | 6.973     | 0.9671    | 6.297     | 626.0     |
| 7         | -4.485e+0 | 12.41     | 0.8322    | 8.286     | 523.2     |
| 8         | -4.485e+0 | 18.81     | 0.3537    | 4.854     | 338.6     |
| 9         | -4.485e+0 | 19.19     | 0.134     | 8.48      | 296.3     |
| 10        | -4.485e+0 | 12.0      | 0.1368    | 8.574     | 332.1     |
| 11        | -4.485e+0 | 17.51     | 0.2784    | 10.03     | 637.5     |
| 12        | -4.485e+0 | 16.36     | 0.2261    | 9.555     | 413.0     |
| 13        | -4.516e+0 | 1.323     | 0.1676    | 10.38     | 161.2     |
| 14        | -4.47e+03 | 6.158     | 

RandomForestRegressor(bootstrap=False, max_depth=5, min_samples_split=10,
                      n_estimators=624)

# neural network

**Feature selection**

In [10]:
# Get the accuracy score of the model with all the features
mlp = MLPRegressor(random_state=0).fit(X_train, y_train)
best_model = mlp
selected_features = X_train.columns.tolist()
best_features = selected_features
best_features_accuracy = cross_val_score(MLPRegressor(random_state=0), X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

while True:
    # Retrieve the learned weights
    weights = mlp.coefs_[0]  # Weights connecting input features to the first hidden layer

    # Normalize the weights
    normalized_weights = np.abs(weights) / np.sum(np.abs(weights), axis=0)

    # Calculate feature importance
    feature_importance = np.mean(normalized_weights, axis=1)

    # Sort features based on their importance
    sorted_indices = np.argsort(feature_importance)
    selected_features = [selected_features[i] for i in sorted_indices[:-1]]

    # When there are no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    mlp = MLPRegressor(random_state=0).fit(X_train[selected_features], y_train)

    # Evaluate Model Performance
    accuracy = cross_val_score(MLPRegressor(random_state=0), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

    # Check if the model performs better without the insignificant features
    if accuracy >= best_features_accuracy:
        best_features_accuracy = accuracy
        best_features = selected_features.copy()
        best_model = mlp

# Save results
best_model_nn = best_model
best_features_nn = best_features

# Show results
print("Best features:", best_features_nn)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_nn)))

Best features: ['NederlandseAntillenEnAruba', 'GeboorteRelatief', 'SterfteRelatief', 'GemiddeldeWoningwaarde', 'PercentageOnbewoond', 'InBezitWoningcorporatie', 'InBezitOverigeVerhuurders', 'EigendomOnbekend', 'BouwjaarVanaf2000', 'kPersonenMetLaagsteInkomen', 'ALandbouwBosbouwEnVisserij', 'KLFinancieleDienstenOnroerendGoed', 'PersonenautoSOverigeBrandstof', 'PersonenautoSPerHuishouden', 'PersonenautoSNaarOppervlakte', 'AfstandTotHuisartsenpraktijk', 'AfstandTotKinderdagverblijf', 'AfstandTotSchool', 'OppervlakteWater', 'TotaalDiefstalUitWoningSchuurED', 'VernielingMisdrijfTegenOpenbareOrde', 'GeweldsEnSeksueleMisdrijven', 'AfstandTotHuisartsenpost', 'AfstandTotApotheek', 'AfstandTotConsultatiebureau', 'AfstandTotOvDagelLevensmiddelen', 'AfstandTotWarenhuis', 'AfstandTotCafeED', 'AfstandTotCafetariaED', 'AfstandTotRestaurant', 'AfstandTotHotelED', 'AfstandTotBuitenschoolseOpvang', 'AfstandTotOpenbaarGroenTotaal', 'AfstandTotDagrecreatiefTerrein', 'AfstandTotBos', 'AfstandTotOpenNatTerr

**Hyperparameter optimalisatie**

In [11]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    regressor = model(**params)
    
    # Calculating rmse based on
    scores = cross_val_score(regressor, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')
    return scores.mean()

params_ranges = {
    'hidden_layer_sizes': (10, 100),
    'alpha': (0.0001, 0.1),
    'learning_rate_init': (0.001, 0.1),
    'max_iter': (100, 1000)
}

# Example usage with Neural Network
model = MLPRegressor
model_bo = BayesianOptimization(f=lambda hidden_layer_sizes, alpha, learning_rate_init, max_iter:
                                    bo_params_generic(model, {
                                        'hidden_layer_sizes': (int(round(hidden_layer_sizes)),),
                                        'alpha': alpha,
                                        'learning_rate_init': learning_rate_init,
                                        'max_iter': int(round(max_iter))
                                    }, X_train[best_features_nn], y_train),
                             pbounds=params_ranges)

results = model_bo.maximize(n_iter=40, init_points=10)
params = model_bo.max['params']

# Creating a model with the best hyperparameters
best_model_nn = model(
    hidden_layer_sizes=(int(round(params['hidden_layer_sizes'])),),
    alpha=params['alpha'],
    learning_rate_init=params['learning_rate_init'],
    max_iter=int(round(params['max_iter']))
)

# Fit the model
best_model_nn.fit(X_train[best_features_nn], y_train)

|   iter    |  target   |   alpha   | hidden... | learni... | max_iter  |
-------------------------------------------------------------------------


| 1         | -67.34    | 0.0966    | 65.79     | 0.07472   | 897.0     |
| 2         | -67.23    | 0.04926   | 43.0      | 0.09549   | 227.9     |
| 3         | -67.02    | 0.0634    | 46.0      | 0.06351   | 320.4     |
| 4         | -67.16    | 0.09866   | 18.07     | 0.06363   | 808.5     |
| 5         | -67.01    | 0.08187   | 88.86     | 0.05362   | 579.4     |
| 6         | -67.19    | 0.03233   | 47.24     | 0.01056   | 100.7     |
| 7         | -67.07    | 0.0709    | 79.64     | 0.09444   | 724.4     |
| 8         | -67.07    | 0.07925   | 26.99     | 0.05911   | 287.5     |
| 9         | -67.12    | 0.01221   | 89.33     | 0.09381   | 610.4     |
| 10        | -67.18    | 0.004295  | 10.5      | 0.09477   | 457.7     |
| 11        | -67.06    | 0.09454   | 45.88     | 0.008566  | 319.4     |
| 12        | -67.12    | 0.08862   | 89.09     | 0.0153    | 582.9     |
| 13        | -67.0     | 0.07129   | 43.5      | 0.06399   | 323.1     |
| 14        | -67.11    | 0.09859   | 

MLPRegressor(alpha=0.0595719792555474, hidden_layer_sizes=(44,),
             learning_rate_init=0.08842356712197524, max_iter=323)

# plotting the bar chart

**using train dataset**

In [12]:
regression_models = [
    {
        'name': 'Multiple Linear Regression',
        'model': best_model_mlr,
        'features': best_features_mlr
    },
    {
        'name': 'Random Forest',
        'model': best_model_rf,
        'features': best_features_rf
    },
    {
        'name': 'Neural Network',
        'model': best_model_nn,
        'features': best_features_nn
    },
    {
        'name': 'Support Vector Machine',
        'model': best_model_svm,
        'features': best_features_svm
    }
]

regression_stats_val = []

for model in regression_models:
    try:
        # Perform cross-validation and calculate predictions and metrics
        y_pred = cross_val_predict(model['model'], X_train[model['features']], y_train, cv=10)
        y_true = y_train
    except: # cv can't be higher then n_splits in the dt
        # Perform cross-validation and calculate predictions and metrics
        y_pred = cross_val_predict(model['model'], X_train[model['features']], y_train, cv=5)
        y_true = y_train

    # R2 Score
    r2 = r2_score(y_true, y_pred)

    # Adjusted R2 Score
    n = len(y_true)  # Number of samples
    p = X_test.shape[1]  # Number of predictors (features) in X
    adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

    # Save stats
    regression_stats_val.append(
        {
            'model': model['name'],
            'mae': mean_absolute_error(y_true, y_pred),
            'mse': mean_squared_error(y_true, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
            'r2': r2,
            'adj_r2': adj_r2
        }
    )

# View stats
regression_stats_val

[{'model': 'Multiple Linear Regression',
  'mae': 51.177004846366174,
  'mse': 4514.103993359065,
  'rmse': 67.18708204230234,
  'r2': 0.015764357653800687,
  'adj_r2': -0.0613873724293279},
 {'model': 'Random Forest',
  'mae': 51.20904367269245,
  'mse': 4482.568726908269,
  'rmse': 66.95198822221988,
  'r2': 0.022640170279610627,
  'adj_r2': -0.05397258233005031},
 {'model': 'Neural Network',
  'mae': 51.47219711855017,
  'mse': 4548.7800670073,
  'rmse': 67.4446444649781,
  'r2': 0.008203737058567473,
  'adj_r2': -0.06954065085397487},
 {'model': 'Support Vector Machine',
  'mae': 51.10157527341273,
  'mse': 4503.4729707350225,
  'rmse': 67.1079203278944,
  'r2': 0.018082299685389658,
  'adj_r2': -0.058887732814076}]

In [13]:
# Extracting model names and metric scores
model_names = [d['model'] for d in regression_stats_val]
metric_labels = list(regression_stats_val[0].keys())[1:]
metric_scores = {metric: [d[metric] for d in regression_stats_val] for metric in metric_labels}

# Plotting the bar chart
fig = go.Figure()

for metric in metric_labels:
    fig.add_trace(go.Bar(
        x=model_names,
        y=metric_scores[metric],
        name=metric
    ))

# Updating the layout
fig.update_layout(
    title='Evaluation Metrics Comparison',
    xaxis_title='Models',
    yaxis_title='Scores',
    barmode='group'
)

# Display the plot
fig.show()

**Evaluating on test data**

In [14]:
regression_models = [
    {
        'name': 'Multiple Linear Regression',
        'model': best_model_mlr,
        'features': best_features_mlr
    },
    {
        'name': 'Random Forest',
        'model': best_model_rf,
        'features': best_features_rf
    },
    {
        'name': 'Neural Network',
        'model': best_model_nn,
        'features': best_features_nn
    },
    {
        'name': 'Support Vector Machine',
        'model': best_model_svm,
        'features': best_features_svm
    }
]

regression_stats_test = []

for model in regression_models:
    # Make predictions on the test set
    y_pred = model['model'].predict(X_test[model['features']])

    # Defining Actual values
    y_true = y_test.copy()

    # R2 Score
    r2 = r2_score(y_true, y_pred)

    # Adjusted R2 Score
    n = len(y_true)  # Number of samples
    p = X_test.shape[1]  # Number of predictors (features) in X
    adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

    # Save stats
    regression_stats_test.append(
        {
            'model': model['name'],
            'mae': mean_absolute_error(y_true, y_pred),
            'mse': mean_squared_error(y_true, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
            'r2': r2,
            'adj_r2': adj_r2
        }
    )

# View stats
regression_stats_test

[{'model': 'Multiple Linear Regression',
  'mae': 50.338805087237034,
  'mse': 4820.445507166995,
  'rmse': 69.42942825032478,
  'r2': 0.03886992011321011,
  'adj_r2': -0.3545751418287506},
 {'model': 'Random Forest',
  'mae': 50.78543381200227,
  'mse': 4848.774530620403,
  'rmse': 69.63314247267894,
  'r2': 0.03322150514110711,
  'adj_r2': -0.36253577345610055},
 {'model': 'Neural Network',
  'mae': 50.52752838528488,
  'mse': 4837.631988326093,
  'rmse': 69.5530875542279,
  'r2': 0.03544317377101891,
  'adj_r2': -0.359404649831488},
 {'model': 'Support Vector Machine',
  'mae': 49.95451175891637,
  'mse': 4780.20192342944,
  'rmse': 69.13900435665414,
  'r2': 0.04689393341137915,
  'adj_r2': -0.34326644472431367}]

In [15]:
# Extracting model names and metric scores
model_names = [d['model'] for d in regression_stats_test]
metric_labels = list(regression_stats_test[0].keys())[1:]
metric_scores = {metric: [d[metric] for d in regression_stats_test] for metric in metric_labels}

# Plotting the bar chart
fig = go.Figure()

for metric in metric_labels:
    fig.add_trace(go.Bar(
        x=model_names,
        y=metric_scores[metric],
        name=metric
    ))

# Updating the layout
fig.update_layout(
    title='Evaluation Metrics Comparison',
    xaxis_title='Models',
    yaxis_title='Scores',
    barmode='group'
)

# Display the plot
fig.show()

Naive bayes

In [16]:
# Defining Actual values
y_true = y_test.copy()

# Defining Predicted values
y_pred = np.full_like(y_test, y_train.mean())

# Mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

# Mean Squared Error
mse = mean_squared_error(y_true, y_pred)
print("Mean Squared Error:", mse)

# Root Mean Squared Error
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

# R2 Score
r2 = r2_score(y_true, y_pred)
print("R2 Score:", r2)

# Adjusted R2 Score
n = len(y_true)  # Number of samples
p = X_test.shape[1]  # Number of predictors (features) in X
adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)
print("Adjusted R2 Score:", adj_r2)

Mean Absolute Error: 50.91615513871266
Mean Squared Error: 5027.712127213051
Root Mean Squared Error: 70.90636168365326
R2 Score: -0.002456173665164929
Adjusted R2 Score: -0.4128183500193261


Save best model as Pickle

In [17]:
import pickle

# Save model
file_path = 'Saved Models/6 MWT.pkl'

with open(file_path, 'wb') as file:
    pickle.dump(best_model_mlr, file)

# Save features
with open('Saved Models/best_features_6 MWT.txt', 'w') as file:
    file.write(f'{best_features_mlr}')